In [2]:
import sys
import jsonpickle
import os
import tweepy
from tweepy import OAuthHandler
import pytz
from pytz import timezone
from tweepy import OAuthHandler
from nltk.tokenize import word_tokenize
from collections import Counter
from nltk.corpus import stopwords
import string
from nltk import trigrams
import re


consumer_key = 'gswlPdHuI7e0qWBAT5OavoH1w'
consumer_secret = 'rvWMe0ldojIsYSUISbcqKxFodIl0VeMGSLf8HOWUcSL6qOywgK'
access_token = '27651816-os3dMfworD2o430p66jdr668XVzvAOnz16YZgAtfj'
access_secret = 'cE8cV46N6qrKzWfuIVogYPaeM003hd2VH6Y38S4T39YGs'
API_KEY = 'gswlPdHuI7e0qWBAT5OavoH1w'
API_SECRET = 'rvWMe0ldojIsYSUISbcqKxFodIl0VeMGSLf8HOWUcSL6qOywgK'
#auth = OAuthHandler(consumer_key, consumer_secret)
#auth.set_access_token(access_token, access_secret)

auth = tweepy.AppAuthHandler(API_KEY, API_SECRET)
api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)


if (not api):
    print ("Can't Authenticate")
    sys.exit(-1)
    
searchQuery = '#traffic'  # this is what we're searching for
maxTweets = 1000 # Some arbitrary large number
tweetsPerQry = 100  # this is the max the API permits
fName = 'tweets.txt' # We'll store the tweets in a text file.


punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['RT', 'rt', 'via', 'I' , 'like']


emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)


emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
        
    )"""


regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]


tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)

stop = stopwords.words('english') + punctuation + ['RT', 'rt', 'via', 'I' , 'like']

def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

eastern = timezone('US/Eastern')
utc = timezone('UTC')

count_all = Counter()
# If results from a specific ID onwards are reqd, set since_id to that ID.
# else default to no lower limit, go as far back as API allows
sinceId = None

# If results only below a specific ID are, set max_id to that ID.
# else default to no upper limit, start from the most recent tweet matching the search query.
max_id = 829208180995530752

tweetCount = 0
print("Downloading max {0} tweets".format(maxTweets))
with open(fName, 'w') as f:
    while tweetCount < maxTweets:
        try:
            if (max_id <= 0):
                if (not sinceId):
                    new_tweets = api.search(q=" ",geocode="35.3302,-80.7325,25km", count=tweetsPerQry)
                else:
                    new_tweets = api.search(q=" ",geocode="35.3302,-80.7325,25km", count=tweetsPerQry,
                                            since_id=sinceId)
            else:
                if (not sinceId):
                    new_tweets = api.search(q=" ",geocode="35.3302,-80.7325,25km", count=tweetsPerQry,
                                            max_id=str(max_id - 1))
                else:
                    new_tweets = api.search(q=" ",geocode="35.3302,-80.7325,25km", count=tweetsPerQry,
                                            max_id=str(max_id - 1),
                                            since_id=sinceId)
            if not new_tweets:
                print("No more tweets found")
                break
            for tweet in new_tweets:
                #if (tweet.geo != None):
                terms_stop = [term for term in preprocess(tweet.text) if term not in stop and not term.startswith(('#', '@')) and not emoji_pattern.search(term)]
                terms_bigram = trigrams(terms_stop)
                count_all.update(terms_bigram)
                f.write(jsonpickle.encode(tweet._json, unpicklable=False) +
                        '\n')
            tweetCount += len(new_tweets)
            if tweetCount % 1000 == 0:
                print("Downloaded {0} tweets".format(tweetCount))
            max_id = new_tweets[-1].id
        except tweepy.TweepError as e:
            # Just exit if any error
            print("some error : " + str(e))
            break

print ("Downloaded {0} tweets, Saved to {1}".format(tweetCount, fName))
print(count_all.most_common(5))


Downloaded 1096 tweets, Saved to tweets.txt
[(("Trump's", 'Travel', 'Ban'), 30), (('Travel', 'Ban', 'Is'), 30), (('Even', 'Associated', 'Press'), 30), (('Blocked', "Trump's", 'Travel'), 30), (('Press', 'Says', 'Judge'), 30)]
